# Start

In [1]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pwd
%cd drive/My Drive/Works/X-ops/ml-ops/zCamp/deploy_batch
!python --version

'/content'

/content/drive/My Drive/Works/X-ops/ml-ops/zCamp/deploy_batch
Python 3.10.12


In [ ]:
# GPU info in Colab
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# 💻 Installation - Environment

In [ ]:
# PyCaret - Low Code ML library (3 to 4 minutes max) --> !! RESTART RUNTIME after installation for $ NEW SESSION only $!!
# %pip install pycaret[full] -q

In [ ]:
# check installed version (must be >3.0)
# import pycaret
# pycaret.__version__

'3.1.0'

In [4]:
# importing required packages $
import pickle
import pandas as pd
import numpy as np

In [8]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


# EDA to Homework-4!!

* [Data Source](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [7]:
# Getting model/saved artifacts into working Environment - 1: $
with open('model.bin', 'rb') as f_in:
  dv, model = pickle.load(f_in)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):

  df = pd.read_parquet(filename)

  df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
  df['duration'] = df.duration.dt.total_seconds() / 60
  df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

  df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

  return df

df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
# Predict using transformed data! - #Q1 - Ans
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

np.std(y_pred)

6.247488852238703

In [17]:
# Preparing Output and Saving results
df['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')
df['preds'] = pd.Series(y_pred)

df_result = df[['ride_id','preds']].copy()
df_result.head()

,ride_id,preds
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [18]:
# Saving results in Parquet format : #Q2 - Ans
df_result.to_parquet('preds.parquet',engine='pyarrow',compression=None,index=False)

# import os
# os.path.getsize('preds.parquet')

68482144